In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
run_root_dir = '../results/dswe/pr_temp_tmin_tmax_dah_trasp_elev_slopeGTOPO_aspectGTOPO_pr_t7_temp_t7_tmin_t7_tmax_t7/'

In [ ]:
save_dir = run_root_dir+'all_k/'

In [ ]:
mode = 'val'   # 'train'

In [ ]:
nfolds = 4

In [ ]:
os.makedirs(save_dir+'train/daily_eval/', exist_ok=True)
os.makedirs(save_dir+'val/daily_eval/', exist_ok=True)
os.makedirs(save_dir+'train/cum_eval/', exist_ok=True)
os.makedirs(save_dir+'val/cum_eval/', exist_ok=True)
os.makedirs(save_dir+'train/swe_eval/', exist_ok=True)
os.makedirs(save_dir+'val/swe_eval/', exist_ok=True)

In [ ]:
def calculate_r2_mae_mse(actual, predicted):
    mse = np.sum((actual - predicted)**2)/len(actual)
    mae = np.mean(abs(predicted - actual))
    r2 = 1 - (mse/np.var(actual))
    return r2, mae, mse

In [ ]:
all_sites_df = pd.read_csv('../data/all_sites_df.csv', index_col=0)

In [ ]:
features = ['pr', 'tmin', 'tmax', 'temp', 'sd', 'swe_t1', 'dah', 'trasp', 'elev', 'elevGTOPO', 'slopeGTOPO', 'aspectGTOPO', 
            'pr_t1', 'temp_t1', 'tmin_t1', 'tmax_t1', 'pr_t2', 'temp_t2', 'tmin_t2', 'tmax_t2', 'pr_t3', 'temp_t3', 'tmin_t3', 'tmax_t3', 
            'pr_t4', 'temp_t4', 'tmin_t4', 'tmax_t4', 'pr_t5', 'temp_t5', 'tmin_t5', 'tmax_t5', 'pr_t6', 'temp_t6', 'tmin_t6', 'tmax_t6', 'pr_t7', 'temp_t7', 'tmin_t7', 'tmax_t7']

In [ ]:
nlags = 7

# SWE Evaluation

Load dataframes including model predictions across folds and remove summer months

In [ ]:
run_swe_dfs_lst = []

In [ ]:
for k in range(nfolds):
    run_fold_cum_dir = f'{run_root_dir}k={k}/{mode}/ALL_SITES_WY_DF.csv'
    run_fold_cum_df = pd.read_csv(run_fold_cum_dir, index_col=0)
    run_swe_dfs_lst.append(run_fold_cum_df)

In [ ]:
run_swe_dfs = pd.concat(run_swe_dfs_lst)

In [ ]:
run_swe_dfs = run_swe_dfs[~pd.to_datetime(run_swe_dfs['datetime']).dt.month.isin([7, 8, 9])]

## Aggregate

Compute performance metrics based on aggregating across all sites and dates.

In [ ]:
swe_UASWE_r2, swe_UASWE_mae, swe_UASWE_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['UASWE'])
swe_LM_r2, swe_LM_mae, swe_LM_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_LM'])
swe_P2M_r2, swe_P2M_mae, swe_P2M_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_P2M'])
swe_P3M_r2, swe_P3M_mae, swe_P3M_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_P3M'])
swe_RF_r2, swe_RF_mae, swe_RF_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_RF'])
swe_NN_r2, swe_NN_mae, swe_NN_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_NN'])

In [ ]:
swe_metrics = {'UA SWE Reg': [swe_UASWE_mse, swe_UASWE_mae, swe_UASWE_r2],
        'Lin Reg': [swe_LM_mse, swe_LM_mae, swe_LM_r2],
        'Quad Reg': [swe_P2M_mse, swe_P2M_mae, swe_P2M_r2],
        'Cub Reg': [swe_P3M_mse, swe_P3M_mae, swe_P3M_r2],
        'RF': [swe_RF_mse, swe_RF_mae, swe_RF_r2],
        'NN': [swe_NN_mse, swe_NN_mae, swe_NN_r2]
       }

In [ ]:
# Creates pandas DataFrame.
swe_metrics_df = pd.DataFrame(swe_metrics, index=['MSE', 'MAE', 'R2'])

In [ ]:
swe_metrics_df

In [ ]:
swe_metrics_df.to_csv(f'{save_dir}{mode}/swe_eval/aggregate_scheme/swe_abl_metrics.csv')

## Climatological

Compute metrics for each day, across sites and years.

In [ ]:
UASWE_metrics = run_swe_dfs[['datetime', 'swe', 'UASWE']].rename(columns={'swe':'Actual', 'UASWE':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day])


In [ ]:
run_noswe_dfs = run_cum_dfs.copy()

In [ ]:
UASWE_metrics = run_swe_dfs[['datetime', 'swe', 'UASWE']].rename(columns={'swe':'Actual', 'UASWE':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
UASWE_metrics.index.names=['Month','Day']
# UASWE_metrics.index.get_level_values('Month')
# UASWE_metrics = UASWE_metrics.drop([7,8,9], level='Month')

LM_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_LM']].rename(columns={'swe':'Actual', 'pred_swe_LM':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
LM_metrics.index.names=['Month','Day']
# LM_metrics.index.get_level_values('Month')
# LM_metrics = LM_metrics.drop([7,8,9], level='Month')

P2M_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_P2M']].rename(columns={'swe':'Actual', 'pred_swe_P2M':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
P2M_metrics.index.names=['Month','Day']
# P2M_metrics.index.get_level_values('Month')
# P2M_metrics = P2M_metrics.drop([7,8,9], level='Month')

P3M_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_P3M']].rename(columns={'swe':'Actual', 'pred_swe_P3M':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
P3M_metrics.index.names=['Month','Day']
# P3M_metrics.index.get_level_values('Month')
# P3M_metrics = P3M_metrics.drop([7,8,9], level='Month')

RF_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_RF']].rename(columns={'swe':'Actual', 'pred_swe_RF':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
RF_metrics.index.names=['Month','Day']
# RF_metrics.index.get_level_values('Month')
# RF_metrics = RF_metrics.drop([7,8,9], level='Month')

NN_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_NN']].rename(columns={'swe':'Actual', 'pred_swe_NN':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
NN_metrics.index.names=['Month','Day']
# NN_metrics.index.get_level_values('Month')
# NN_metrics = NN_metrics.drop([7,8,9], level='Month')

In [ ]:
UASWE_metrics['date'] = UASWE_metrics.index.get_level_values('Month').astype(str) + '-' + UASWE_metrics.index.get_level_values('Day').astype(str)
LM_metrics['date'] = LM_metrics.index.get_level_values('Month').astype(str) + '-' + LM_metrics.index.get_level_values('Day').astype(str)
P2M_metrics['date'] = P2M_metrics.index.get_level_values('Month').astype(str) + '-' + P2M_metrics.index.get_level_values('Day').astype(str)
P3M_metrics['date'] = P3M_metrics.index.get_level_values('Month').astype(str) + '-' + P3M_metrics.index.get_level_values('Day').astype(str)
RF_metrics['date'] = RF_metrics.index.get_level_values('Month').astype(str) + '-' + RF_metrics.index.get_level_values('Day').astype(str)
NN_metrics['date'] = NN_metrics.index.get_level_values('Month').astype(str) + '-' + NN_metrics.index.get_level_values('Day').astype(str)

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['mse'], UASWE_metrics[:181]['mse']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['mse'], LM_metrics[:181]['mse']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['mse'], P2M_metrics[:181]['mse']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['mse'], P3M_metrics[:181]['mse']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['mse'], RF_metrics[:181]['mse']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['mse'], NN_metrics[:181]['mse']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('SWE: MSE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_dir}{mode}/swe_eval/climatological_scheme/swe_mse_climat.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['rmse'], UASWE_metrics[:181]['rmse']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['rmse'], LM_metrics[:181]['rmse']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['rmse'], P2M_metrics[:181]['rmse']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['rmse'], P3M_metrics[:181]['rmse']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['rmse'], RF_metrics[:181]['rmse']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['rmse'], NN_metrics[:181]['rmse']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('SWE: RMSE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
ax.set_ylim(0,200)
# plt.savefig(f'{save_dir}{mode}/swe_eval/climatological_scheme/swe_rmse_climat.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['mae'], UASWE_metrics[:181]['mae']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['mae'], LM_metrics[:181]['mae']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['mae'], P2M_metrics[:181]['mae']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['mae'], P3M_metrics[:181]['mae']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['mae'], RF_metrics[:181]['mae']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['mae'], NN_metrics[:181]['mae']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('SWE: MAE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_dir}{mode}/swe_eval/climatological_scheme/swe_mae_climat.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['r2'], UASWE_metrics[:181]['r2']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['r2'], LM_metrics[:181]['r2']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['r2'], P2M_metrics[:181]['r2']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['r2'], P3M_metrics[:181]['r2']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['r2'], RF_metrics[:181]['r2']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['r2'], NN_metrics[:181]['r2']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('SWE: R2')
ax.legend()
ax.set_ylim(-1, 1)
ax.axhline(0, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_dir}{mode}/swe_eval/climatological_scheme/swe_r2_climat.png', dpi=300)
plt.show()

## Site-Year

Compute metrics by site-year.

In [ ]:
mse_swe_UASWE, mse_swe_LM, mse_swe_P2M, mse_swe_P3M, mse_swe_RF, mse_swe_NN = [], [], [], [], [], []
mae_swe_UASWE, mae_swe_LM, mae_swe_P2M, mae_swe_P3M, mae_swe_RF, mae_swe_NN = [], [], [], [], [], []
r2_swe_UASWE, r2_swe_LM, r2_swe_P2M, r2_swe_P3M, r2_swe_RF, r2_swe_NN = [], [], [], [], [], []

In [ ]:
for site in np.unique(run_swe_dfs['sitecode']):
    for wy in np.unique(run_swe_dfs['water_year']):
        site_wy_df = run_swe_dfs[(run_swe_dfs['sitecode'] == site) & (run_swe_dfs['water_year'] == wy)]
        
        if not site_wy_df.empty:
            
            mse_swe_UASWE.append(mean_squared_error(site_wy_df['swe'], site_wy_df['UASWE'], squared=True))
            mse_swe_LM.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_LM'], squared=True))
            mse_swe_P2M.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_P2M'], squared=True))
            mse_swe_P3M.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_P3M'], squared=True))
            mse_swe_RF.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_RF'], squared=True))
            mse_swe_NN.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_NN'], squared=True))     
            mae_swe_UASWE.append(np.mean(abs(site_wy_df['UASWE'] - site_wy_df['swe'])))
            mae_swe_LM.append(np.mean(abs(site_wy_df['pred_swe_LM'] - site_wy_df['swe'])))
            mae_swe_P2M.append(np.mean(abs(site_wy_df['pred_swe_P2M'] - site_wy_df['swe'])))
            mae_swe_P3M.append(np.mean(abs(site_wy_df['pred_swe_P3M'] - site_wy_df['swe'])))
            mae_swe_RF.append(np.mean(abs(site_wy_df['pred_swe_RF'] - site_wy_df['swe'])))
            mae_swe_NN.append(np.mean(abs(site_wy_df['pred_swe_NN'] - site_wy_df['swe'])))
            r2_swe_UASWE.append(r2_score(site_wy_df['swe'], site_wy_df['UASWE']))
            r2_swe_LM.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_LM']))
            r2_swe_P2M.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_P2M']))
            r2_swe_P3M.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_P3M']))
            r2_swe_RF.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_RF']))
            r2_swe_NN.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_NN']))


In [ ]:
swe_metrics_med = {'UA SWE Reg': [np.median(mse_swe_UASWE), np.median(mae_swe_UASWE), np.median(r2_swe_UASWE)],
                   'Lin Reg': [np.median(mse_swe_LM), np.median(mae_swe_LM), np.median(r2_swe_LM)],
                   'Quad Reg': [np.median(mse_swe_P2M), np.median(mae_swe_P2M), np.median(r2_swe_P2M)],
                   'Cub Reg': [np.median(mse_swe_P3M), np.median(mae_swe_P3M), np.median(r2_swe_P3M)],
                   'RF': [np.median(mse_swe_RF), np.median(mae_swe_RF), np.median(r2_swe_RF)],
                   'NN': [np.median(mse_swe_NN), np.median(mae_swe_NN), np.median(r2_swe_NN)]
                  }
swe_metrics_mean = {'UA SWE Reg': [np.mean(mse_swe_UASWE), np.mean(mae_swe_UASWE), np.mean(r2_swe_UASWE)],
               'Lin Reg': [np.mean(mse_swe_LM), np.mean(mae_swe_LM), np.mean(r2_swe_LM)],
               'Quad Reg': [np.mean(mse_swe_P2M), np.mean(mae_swe_P2M), np.mean(r2_swe_P2M)],
               'Cub Reg': [np.mean(mse_swe_P3M), np.mean(mae_swe_P3M), np.mean(r2_swe_P3M)],
               'RF': [np.mean(mse_swe_RF), np.mean(mae_swe_RF), np.mean(r2_swe_RF)],
               'NN': [np.mean(mse_swe_NN), np.mean(mae_swe_NN), np.mean(r2_swe_NN)]
              }
swe_metrics_std = {'UA SWE Reg': [np.std(mse_swe_UASWE), np.std(mae_swe_UASWE), np.std(r2_swe_UASWE)],
               'Lin Reg': [np.std(mse_swe_LM), np.std(mae_swe_LM), np.std(r2_swe_LM)],
               'Quad Reg': [np.std(mse_swe_P2M), np.std(mae_swe_P2M), np.std(r2_swe_P2M)],
               'Cub Reg': [np.std(mse_swe_P3M), np.std(mae_swe_P3M), np.std(r2_swe_P3M)],
               'RF': [np.std(mse_swe_RF), np.std(mae_swe_RF), np.std(r2_swe_RF)],
               'NN': [np.std(mse_swe_NN), np.std(mae_swe_NN), np.std(r2_swe_NN)]
              }

In [ ]:
swe_abl_means = pd.DataFrame(swe_metrics_mean, index=['MSE','MAE','R2'])

In [ ]:
swe_abl_meds = pd.DataFrame(swe_metrics_med, index=['MSE','MAE','R2'])

In [ ]:
swe_abl_stds = pd.DataFrame(swe_metrics_std, index=['MSE','MAE','R2'])

In [ ]:
swe_abl_means.to_csv(f'{save_dir}{mode}/swe_eval/site_year_scheme/swe_abl_means.csv')
swe_abl_meds.to_csv(f'{save_dir}{mode}/swe_eval/site_year_scheme/swe_abl_meds.csv')
swe_abl_stds.to_csv(f'{save_dir}{mode}/swe_eval/site_year_scheme/swe_abl_stds.csv')

In [ ]:
swe_abl_means

In [ ]:
swe_abl_meds

# Climatological SWE Time Series

Generate climatological time series of SWE model predictions and compare it to that of SNOTEL SWE.

In [ ]:
dfs_sub = run_swe_dfs[['datetime', 'swe', 'UASWE', 'pred_swe_UASWE',
       'pred_swe_LM', 'pred_swe_P2M', 'pred_swe_P3M', 'pred_swe_RF',
       'pred_swe_NN']]

In [ ]:
snotel_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['swe'].mean()
uaswe_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['UASWE'].mean()
myswe_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_UASWE'].mean()
lm_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_LM'].mean()
p2m_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_P2M'].mean()
p3m_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_P3M'].mean()
rf_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_RF'].mean()
nn_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_NN'].mean()

In [ ]:
# print(lm_swe_mean, p2m_swe_mean, p3m_swe_mean, rf_swe_mean)

In [ ]:
snotel_swe_mean.index.names = ['Month', 'Day']
uaswe_swe_mean.index.names = ['Month', 'Day']
myswe_swe_mean.index.names = ['Month', 'Day']
lm_swe_mean.index.names = ['Month', 'Day']
p2m_swe_mean.index.names = ['Month', 'Day']
p3m_swe_mean.index.names = ['Month', 'Day']
rf_swe_mean.index.names = ['Month', 'Day']
nn_swe_mean.index.names = ['Month', 'Day']

In [ ]:
snotel_swe_mean = snotel_swe_mean.to_frame()
uaswe_swe_mean = uaswe_swe_mean.to_frame()
myswe_swe_mean = myswe_swe_mean.to_frame()
lm_swe_mean = lm_swe_mean.to_frame()
p2m_swe_mean = p2m_swe_mean.to_frame()
p3m_swe_mean = p3m_swe_mean.to_frame()
rf_swe_mean = rf_swe_mean.to_frame()
nn_swe_mean = nn_swe_mean.to_frame()

In [ ]:
snotel_swe_mean['date'] = snotel_swe_mean.index.get_level_values('Month').astype(str) + '-' + snotel_swe_mean.index.get_level_values('Day').astype(str)
uaswe_swe_mean['date'] = uaswe_swe_mean.index.get_level_values('Month').astype(str) + '-' + uaswe_swe_mean.index.get_level_values('Day').astype(str)
myswe_swe_mean['date'] = myswe_swe_mean.index.get_level_values('Month').astype(str) + '-' + myswe_swe_mean.index.get_level_values('Day').astype(str)
lm_swe_mean['date'] = lm_swe_mean.index.get_level_values('Month').astype(str) + '-' + lm_swe_mean.index.get_level_values('Day').astype(str)
p2m_swe_mean['date'] = p2m_swe_mean.index.get_level_values('Month').astype(str) + '-' + p2m_swe_mean.index.get_level_values('Day').astype(str)
p3m_swe_mean['date'] = p3m_swe_mean.index.get_level_values('Month').astype(str) + '-' + p3m_swe_mean.index.get_level_values('Day').astype(str)
rf_swe_mean['date'] = rf_swe_mean.index.get_level_values('Month').astype(str) + '-' + rf_swe_mean.index.get_level_values('Day').astype(str)
nn_swe_mean['date'] = nn_swe_mean.index.get_level_values('Month').astype(str) + '-' + nn_swe_mean.index.get_level_values('Day').astype(str)

In [ ]:
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([snotel_swe_mean[181:]['date'], snotel_swe_mean[:181]['date']]).values, pd.concat([snotel_swe_mean[181:]['swe'], snotel_swe_mean[:181]['swe']]).values, label='SNOTEL SWE', color='black')
ax.plot(pd.concat([uaswe_swe_mean[181:]['date'], uaswe_swe_mean[:181]['date']]).values, pd.concat([uaswe_swe_mean[181:]['UASWE'], uaswe_swe_mean[:181]['UASWE']]).values, label='UA SWE', color='grey')
ax.plot(pd.concat([myswe_swe_mean[181:]['date'], myswe_swe_mean[:181]['date']]).values, pd.concat([myswe_swe_mean[181:]['pred_swe_UASWE'], myswe_swe_mean[:181]['pred_swe_UASWE']]).values, label='UA SWE [MY]')
ax.plot(pd.concat([lm_swe_mean[181:]['date'], lm_swe_mean[:181]['date']]).values, pd.concat([lm_swe_mean[181:]['pred_swe_LM'], lm_swe_mean[:181]['pred_swe_LM']]).values, label='LM-derived SWE')
ax.plot(pd.concat([p2m_swe_mean[181:]['date'], p2m_swe_mean[:181]['date']]).values, pd.concat([p2m_swe_mean[181:]['pred_swe_P2M'], p2m_swe_mean[:181]['pred_swe_P2M']]).values, label='P2M-derived SWE')
ax.plot(pd.concat([p3m_swe_mean[181:]['date'], p3m_swe_mean[:181]['date']]).values, pd.concat([p3m_swe_mean[181:]['pred_swe_P3M'], p3m_swe_mean[:181]['pred_swe_P3M']]).values, label='P3M-derived SWE')
ax.plot(pd.concat([rf_swe_mean[181:]['date'], rf_swe_mean[:181]['date']]).values, pd.concat([rf_swe_mean[181:]['pred_swe_RF'], rf_swe_mean[:181]['pred_swe_RF']]).values, label='RF-derived SWE')
ax.plot(pd.concat([nn_swe_mean[181:]['date'], nn_swe_mean[:181]['date']]).values, pd.concat([nn_swe_mean[181:]['pred_swe_NN'], nn_swe_mean[:181]['pred_swe_NN']]).values, label='NN-derived SWE')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([snotel_swe_mean[181:]['date'], snotel_swe_mean[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Mean SWE on Each Date Across Sites and Years')
ax.set_ylabel('Mean SWE (mm)')
ax.set_xlabel('Date')
ax.legend()
# ax.set_ylim(0,700)
# ax.axhline(200, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_dir}{mode}/mean_swe_ts.png', dpi=300)